In [1]:
import numpy as np
import  matplotlib.pyplot as plt
import torch
import torchaudio

## Test model

In [ ]:
model_script_path = "./fine_tuned.pt"
model_script_path = "./general.pt"
model_script_path = "./piezo_only.pt"

model = torch.jit.load(model_script_path).cuda()

folder = './roughness_dataset/opus/stereo/'
contact_threshold = 0.005

smooth_stones_test = [21]
rough_stones_test = [22]

In [ ]:
def test_file(folder, file, label, x):
    
    wave, _ = torchaudio.load(folder+file)
    wave = torchaudio.functional.resample(wave, RESAMPLE_FACTOR, 1)

    parts = cut_stepped(wave)
    loudness = parts[:,0].abs().mean(dim=1)
    loud_parts_mask = loudness > contact_threshold

    with torch.inference_mode():
        pred = model(parts.cuda())[:,2].exp()
    
    loss = 0.0
    acc = 0.0
    
    if label == 1:
        off = (label - pred)[loud_parts_mask]
        if off.shape[0] == 0:
            print(file+": no chunks above threshold!")
            return 0, 0, 0, label, x
        loss = off.abs().mean().item()
        acc = float((off <= 0.5).sum()) / float(off.shape[0])
    
    else:
        off = torch.abs((label - pred)[loud_parts_mask])
        if off.shape[0] == 0:
            print(file+": no chunks above threshold!")
            return 0, 0, 0, label, x
        loss = off.abs().mean().item()
        acc = float((off < 0.5).sum()) / float(off.shape[0])
    
    fig = plt.figure(figsize=(17,5))
    
   
    
    if label == 0:
        color = "blue"
    else:
        color = "orange"
    
    ax = fig.add_subplot(2, 1, 1)
    #ax.plot(pred[loud_parts_mask].cpu().numpy(),color=color)
    ax.plot(pred.cpu().numpy(),color=color)
    ax.set_title(file+" (Roughness confidence)")
    ax.set_ylim(0,1)
    
    ax = fig.add_subplot(2, 1, 2)
    #ax.plot(wave[0,511:][loud_parts_mask].cpu().numpy(),color=color)
    ax.plot(wave[0,511:].cpu().numpy(),color=color)
    ax.set_title(file+" (Audio)")
    ax.set_ylim(-1,1)
    
    fig.tight_layout()
    
    print(file+"    \tLoss: "+str(np.round(loss,4))+" \tAccuracy: "+str(np.round(acc,4))+" \tUsed chunks: "+str(int(loud_parts_mask.float().sum().item())))
    
    return loss, acc, float(off.shape[0]), label, x

In [ ]:
def test_files(folder,file_ids,label):
    print("\n#################################################\nTesting "+("smooth" if label==0 else "rough")+" objects:\n")
    
    total_counts = []
    avg_loss = []
    avg_acc = []
        
    for x in file_ids:
        if x < 10:
            x = "0"+str(x)
        else:
            x = str(x)
        
        print("Testing object '"+folder+str(x)+"_' with label '"+str(label)+"'...")
        
        results = []
        results.append(test_file(folder,x+'_long_light.wav',label,int(x)))
        results.append(test_file(folder,x+'_long_medium.wav',label,int(x)))
        results.append(test_file(folder,x+'_long_loud.wav',label,int(x)))
        results.append(test_file(folder,x+'_short_light.wav',label,int(x)))
        results.append(test_file(folder,x+'_short_medium.wav',label,int(x)))
        results.append(test_file(folder,x+'_short_loud.wav',label,int(x)))
        results.append(test_file(folder,x+'_wiggle.wav',label,int(x)))

        res = np.array(results)
        
        avg_loss.append(np.sum(res[:,0]*(res[:,2]/np.sum(res[:,2]))))
        avg_acc.append(np.sum(res[:,1]*(res[:,2]/np.sum(res[:,2]))))
        total_counts.append(int(np.sum(res[:,2])))
        
        print("---")
        print("Avg. loss of object '"+str(x)+"': \t"+str(np.round(avg_loss[-1],4)))
        print("Avg. accuracy of object '"+str(x)+"': \t"+str(np.round(avg_acc[-1],4)))
        print("Total chunks of object '"+str(x)+"': \t"+str(total_counts[-1])+"\n")
    
    avg_loss = np.asarray(avg_loss)
    avg_acc = np.asarray(avg_acc)
    total_counts = np.asarray(total_counts)
    avg_loss_mean = np.sum(avg_loss*(total_counts/np.sum(total_counts)))
    avg_acc_mean = np.sum(avg_acc*(total_counts/np.sum(total_counts)))
    total_counts_mean = np.sum(total_counts)
        
    print("------")
    print("Avg. loss of all "+("smooth" if label==0 else "rough")+" objects: \t"+str(np.round(avg_loss_mean,4)))
    print("Avg. accuracy of all "+("smooth" if label==0 else "rough")+" objects: \t"+str(np.round(avg_acc_mean,4)))
    print("Total chunks of all "+("smooth" if label==0 else "rough")+" objects: \t"+str(total_counts_mean))
    
    return avg_loss_mean, avg_acc_mean, total_counts_mean


smooth_stats = test_files(folder,smooth_stones_test,0)
rough_stats = test_files(folder,rough_stones_test,1)

print("\n#################################################\nConfusion Matrix:\n")
chunks_total = smooth_stats[2]+rough_stats[2]

fp = (1-smooth_stats[1]) * smooth_stats[2] / chunks_total
tp = rough_stats[1] * rough_stats[2] / chunks_total
fn = (1-rough_stats[1]) * rough_stats[2] / chunks_total
tn = smooth_stats[1] * smooth_stats[2] / chunks_total

print("fp",fp,"tp",tp,"fn",fn,"tn",tn)
print("sum:",np.sum([fp,tp,fn,tn]),"chunks_total",chunks_total)

print("\n#################################################\nTesting final runs:\n")

smooth_runs = []
smooth_runs.append(test_file("./roughness_dataset/finals_recordings/","day_1_smooth.wav",0,None))
smooth_runs.append(test_file("./roughness_dataset/finals_recordings/","day_2_smooth.wav",0,None))
smooth_runs = np.asarray(smooth_runs)

rough_runs = []
rough_runs.append(test_file("./roughness_dataset/finals_recordings/","day_1_rough.wav",1,None))
rough_runs.append(test_file("./roughness_dataset/finals_recordings/","day_2_rough.wav",1,None))
rough_runs = np.asarray(rough_runs)

rough_chunks = np.sum(rough_runs[:,2])
smooth_chunks = np.sum(smooth_runs[:,2])
total_chunks = rough_chunks+smooth_chunks

rough_loss = np.sum(rough_runs[:,0] * (rough_runs[:,2] / rough_chunks))
rough_accuracy = np.sum(rough_runs[:,1] * (rough_runs[:,2] / rough_chunks))
print("\nrough_loss",rough_loss,"\trough_accuracy",rough_accuracy)

smooth_loss = np.sum(smooth_runs[:,0] * (smooth_runs[:,2] / smooth_chunks))
smooth_accuracy = np.sum(smooth_runs[:,1] * (smooth_runs[:,2] / smooth_chunks))
print("smooth_loss",smooth_loss,"\tsmooth_accuracy",smooth_accuracy)

print("\n#################################################\nConfusion Matrix:\n")

fp = (1-smooth_accuracy) * smooth_chunks / total_chunks
tp = rough_accuracy * rough_chunks / total_chunks
fn = (1-rough_accuracy) * rough_chunks / total_chunks
tn = smooth_accuracy * smooth_chunks / total_chunks

print("fp",fp,"tp",tp,"fn",fn,"tn",tn)
print("sum:",np.sum([fp,tp,fn,tn]),"total_chunks",total_chunks)